# **Problem Statement**  
## **32. Design a data structure for autocomplete system.**

Design a data structure that suggests up to the top 3 most frequently typed sentences starting with the current input prefix.

The system should support:

- input(char c):
    - If c != '#', return the top 3 sentences that have the current prefix (sorted by frequency, then lexicographically).
    - If c == '#', store the current prefix as a new sentence in the system (frequency +1), then reset the prefix.

The goal is to simulate a real-time autocomplete engine similar to how typing works in a search bar.

### Constraints & Example Inputs/Outputs

- Maximum number of stored sentences: ≤ 10⁴
- Each sentence length ≤ 100
- The system should return results within milliseconds (near real-time suggestion).

### Example 1:
```python 
sentences = ["i love you", "island", "ironman", "i love leetcode"]
times = [5, 3, 2, 2]

autocomplete = AutocompleteSystem(sentences, times)
print(autocomplete.input('i'))   # → ["i love you", "island", "i love leetcode"]
print(autocomplete.input(' '))   # → ["i love you", "i love leetcode"]
print(autocomplete.input('a'))   # → []
print(autocomplete.input('#'))   # → []


### Solution Approach

#### Core Idea:
We use a Trie (Prefix Tree) where:
- Each node stores:
    - children → next characters.
    - counts → a map {sentence: frequency} for quick lookup.

When an User types a character:
1. Append it to the current prefix.
2. Traverse the Trie to find the prefix node.
3. Gather all sentences starting from that prefix.
4. Sort by:
   - Frequency(descending)
   - Lexicographical Order(ascending)
5. Return top 3 suggestions.

when '#' is entered:
- Insert/Update the sentence in the Trie (increase frequency).
- Reset Prefix.

### Solution Code

In [2]:
# Approach1: Brute Force Approach (For conceptual baseline comparison - not efficient for large data)
class AutocompleteBruteForce:
    def __init__(self, sentences, times):
        self.freq = {s: t for s, t in zip(senstences, times)}
        self.prefix = ""

    def input(self, c):
        if c == "#":
            self.freq[self.prefix] = self.freq.get(self.prefix, 0) + 1
            self.prefix = ""
            return []
        self.prefix += c
        candidates = [s for s in self.freq if s.startswith(self.prefix)]
        candidates.sort(key=lambda s: (-self.freq[s], s))
        return candidates[:3]

### Time Complexity:
- For each input: O(N * L log N)
(where N = total sentences, L = avg sentence length)

This works fine for small input but not scalable.

### Alternative Solution

In [3]:
# Approach2: Optimized Approach (Trie-based Autocomplete System)
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = {}
        self.counts = defaultdict(int)

class AutocompleteSystem:
    def __init__(self, sentences, times):
        self.root = TrieNode()
        self.keyword = ""
        for s, t in zip(sentences, times):
            self.add_sentence(s, t)

    def add_sentence(self, sentence, count):
        node = self.root
        for ch in sentence:
            if ch not in node.children:
                node.children[ch] = TrieNode()
            node = node.children[ch]
            node.counts[sentence] += count

    def search(self, prefix):
        node = self.root
        for ch in prefix:
            if ch not in node.children:
                return []
            node = node.children[ch]
        return sorted(node.counts.keys(), key=lambda s: (-node.counts[s], s))[:3]

    def input(self, c):
        if c == '#':
            self.add_sentence(self.keyword, 1)
            self.keyword = ""
            return []
        self.keyword += c
        return self.search(self.keyword)

### Alternative Approaches

| Approach                     | Description                  | Pros                    | Cons                    |
| ---------------------------- | ---------------------------- | ----------------------- | ----------------------- |
| **Brute Force (list-based)** | Linear scan and prefix match | Simple                  | Too slow for real-time  |
| **Trie (current)**           | Prefix-indexed lookup        | Fast & memory efficient | Slightly complex        |
| **Ternary Search Tree**      | Memory-optimized Trie        | Space-saving            | Harder to implement     |
| **Prefix Hash Map**          | Stores top-k for each prefix | Instant results         | High preprocessing time |


### Test Cases 

In [4]:
def test_autocomplete_system():
    sentences = ["i love you", "island", "ironman", "i love leetcode"]
    times = [5, 3, 2, 2]
    ac = AutocompleteSystem(sentences, times)

    print(ac.input('i'))   # ["i love you", "island", "i love leetcode"]
    print(ac.input(' '))   # ["i love you", "i love leetcode"]
    print(ac.input('a'))   # []
    print(ac.input('#'))   # []

    # Test adding a new sentence
    ac.input('i')
    ac.input(' ')
    ac.input('a')
    ac.input('#')  # Adds "i a" with frequency 1

    print(ac.input('i'))   # ["i love you", "island", "i love leetcode"]
    print(ac.input(' '))   # ["i love you", "i love leetcode", "i a"]

test_autocomplete_system()


['i love you', 'island', 'i love leetcode']
['i love you', 'i love leetcode']
[]
[]
['i love you', 'island', 'i a']
['i love you', 'i a', 'i love leetcode']


## Complexity Analysis

| Operation        | Time Complexity | Space Complexity | Description                                 |
| ---------------- | --------------- | ---------------- | ------------------------------------------- |
| `add_sentence()` | O(L)            | O(L * N)         | Insert sentence into Trie                   |
| `search(prefix)` | O(P + k log k)  | —                | P = prefix length, k = #results             |
| `input()`        | O(L log L)      | —                | Sort top results                            |
| Overall          | **O(L log L)**  | **O(N * L)**     | Very efficient for interactive autocomplete |


### Real World Analogy

This structure is how:
- Google autocomplete predicts what you’ll type next.
- Spotify / YouTube search bars suggest most played items.
- Code editors (VSCode / Copilot) autocomplete identifiers and function names.

#### Thank You!!